In [1]:
#Import d'outils pythons
import random
import operator
import math
import pandas as pd
import numpy 

# 1. On commence par récupéres les données 

In [2]:
#Import dataset  de la Bourse de Paris
dataParis =  pd.read_csv("^FCHI.csv")
#Import dataset de la Bourse de Francfort
dataFrancfort = pd.read_csv("^GDAXI.csv")
#Import dataset  de la Bourse de NewYork
dataNY =  pd.read_csv("^DJI.csv")

#Import dataset  de la Bourse de societe Hi-tech
dataHiTech =  pd.read_csv("^IXIC.csv")

# 2. Ne prenez en compte que la colonne Adj Close ; Remplissez les données manquantes en recopiant à chaque fois la donnée disponible la plus récente.

In [3]:
#On garde seulement la colonne "Adj CLose"
adjParis = dataParis["Adj Close"]
#On a qu'une seule valeur manquante en position 30 donc on v la remplacer par celle en position 31

#On garde seulement la colonne "Adj CLose"
adjNY = dataNY["Adj Close"]
#Il n'y a pas de valeur manquante donc on continue sans rien changer

##On verifie la longueur des deux dataset choisit 
print(len(dataNY))
print(len(dataParis))

253
258


In [4]:
##On voit que les 2 dataset n'ont pas le même nombre de lignes donc pour les prochaines calcules ont va fusionner nos dataframe avec la date et remplir les trous
adjNYDate = dataNY[["Date","Adj Close"]]
adjParisDate = dataParis[["Date","Adj Close"]]
adjFCDate = dataFrancfort[["Date","Adj Close"]]
adjHiTechDate = dataHiTech[["Date","Adj Close"]]

# ON renomme les colonnes pour savoir a qui correspond les données
adjNYDate = adjNYDate.rename(columns={"Adj Close": "NY Adj Close"})
adjParisDate = adjParisDate.rename(columns={"Adj Close": "Paris Adj Close"})
adjFCDate = adjFCDate.rename(columns={"Adj Close": "FC Adj Close"})
adjHiTechDate = adjHiTechDate.rename(columns={"Adj Close": "HT Adj Close"})

adjParisNY = pd.merge(adjParisDate, adjNYDate, how='outer')
adjHTFC = pd.merge(adjHiTechDate,adjFCDate, how='outer')
adjAll = pd.merge(adjHTFC,adjParisNY, how='outer')

In [48]:
adjAll.head()

,Date,HT Adj Close,FC Adj Close,Paris Adj Close,NY Adj Close
0,2019-11-13,8482.099609,13221.120117,5907.089844,27783.589844
1,2019-11-14,8479.019531,13241.750000,5901.080078,27781.960938
2,2019-11-15,8540.830078,13241.750000,5939.270020,28004.890625
3,2019-11-18,8549.940430,13207.009766,5929.790039,28036.220703
4,2019-11-19,8570.660156,13221.120117,5909.049805,27934.019531


In [6]:
##On remplace les NaN avec les valeurs au alentour
def isNan(data, column):
    for i in range(len(data)-1):
        if math.isnan(data[column].loc[i]) :
            if i == 0 :
                data[column].loc[i] = data[column].loc[i+4]
            elif i < len(data):
                data[column].loc[i] = data[column].loc[i+1]
            else : 
                data[column].loc[i] = data[column].loc[i-1]
    return data
##On applique la fct
adjAll = isNan(adjAll,"Paris Adj Close")
adjAll = isNan(adjAll,"NY Adj Close")
adjAll = isNan(adjAll,"FC Adj Close")
adjAll = isNan(adjAll,"HT Adj Close")

##On fait des dataset contenant que la colonne qui nous interesse
adjParis = adjAll["Paris Adj Close"]
adjNY = adjAll["NY Adj Close"]
adjFC = adjAll["FC Adj Close"]
adjHT = adjAll["HT Adj Close"]

/home/deptinfo/.local/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# 3. Définissez une série discrète sur la base de chacune des séries ainsi obtenues en utilisant la transformation ci-dessous. Soit xt la valeur d’Adj Close pour la période t = {0, 1, . . .} ; on va calculer le terme de la série discrète pour la période t = {1, . . .}, st , comme suit : <img src="q3.png">

In [7]:
# +1 signifie que la valeur de l’indice a augmenté, −1 qu’elle a diminué et 0 qu’il
# n’y a pas eu de mouvement digne de note
def createSerie(adj):
    serie = []

    for i in range(len(adj)):
        if i == 0 :
            serie.append(0)
        else :
            ln = math.log(adj[i]/adj[i-1])
            if ln > 0.005 :
                serie.append(1)
            elif ln <= 0.005 and ln >= -0.005 :
                serie.append(0)
            else :
                serie.append(-1)
    return serie

serieParis = createSerie(adjParis)
serieNY = createSerie(adjNY)
serieFC = createSerie(adjFC)
serieHT = createSerie(adjHT)

# 4. Obtenez la table des probabilités des st pour chacun des indices considérés.

In [8]:
#On calcule la probabilité de chaque cas de figure (1,0,-1)
def probSerie(serie):
    prob = []
    prob.append(serie.count(1)/len(serie))
    prob.append(serie.count(0)/len(serie))
    prob.append(serie.count(-1)/len(serie))
    print("On a p = ",prob[0]," d'avoir un indice qui augumente (+1)")
    print("On a p = ",prob[1]," d'avoir un indice quasi inchangé (0)")
    print("On a p = ",prob[2]," d'avoir un indice qui baisse (-1)")
    return prob

In [9]:
print("Probabilité appliqué a la serie de Paris")
probParis = probSerie(serieParis)

Probabilité appliqué a la serie de Paris
On a p =  0.34099616858237547  d'avoir un indice qui augumente (+1)
On a p =  0.36015325670498083  d'avoir un indice quasi inchangé (0)
On a p =  0.2988505747126437  d'avoir un indice qui baisse (-1)


In [10]:
print("Probabilité appliqué a la serie de NeyWork")
probNY = probSerie(serieNY)

Probabilité appliqué a la serie de NeyWork
On a p =  0.3563218390804598  d'avoir un indice qui augumente (+1)
On a p =  0.34099616858237547  d'avoir un indice quasi inchangé (0)
On a p =  0.30268199233716475  d'avoir un indice qui baisse (-1)


In [11]:
print("Probabilité appliqué a la serie de Francfort")
probFC= probSerie(serieFC)

Probabilité appliqué a la serie de Francfort
On a p =  0.36015325670498083  d'avoir un indice qui augumente (+1)
On a p =  0.3486590038314176  d'avoir un indice quasi inchangé (0)
On a p =  0.29118773946360155  d'avoir un indice qui baisse (-1)


In [12]:
print("Probabilité appliqué a la serie de Hi Tech")
probHT = probSerie(serieHT)

Probabilité appliqué a la serie de Hi Tech
On a p =  0.42911877394636017  d'avoir un indice qui augumente (+1)
On a p =  0.27586206896551724  d'avoir un indice quasi inchangé (0)
On a p =  0.2950191570881226  d'avoir un indice qui baisse (-1)


# 5. Calculez, pour chaque indice : <img src="q5.png">

## **Fct pour calculer Pr(st,st+1) :**

In [13]:
def stvoisin(data):
    couple00 = 0
    couple01 = 0
    couple11 = 0
    couple10 = 0
    coupleN10 = 0
    couple0N1 = 0
    coupleN1N1 = 0
    coupleN11 = 0
    couple1N1 = 0
    indicecouple = []
    for i in range (len(data)):
        if i != 0 :
            if data[i - 1] == 0 and data[i] == 0:
                couple00 += 1
            elif data[i - 1] == 0 and data[i] == 1:
                couple01 += 1
            elif data[i - 1] == 1 and data[i] == 1:
                couple11 += 1
            elif data[i - 1] == 1 and data[i] == 0:
                couple10 += 1
            elif data[i - 1] == 0 and data[i] == -1:
                couple0N1 += 1
            elif data[i - 1] == -1 and data[i] == -1:
                coupleN1N1 += 1
            elif data[i - 1] == -1 and data[i] == 0:
                coupleN10 += 1
            elif data[i - 1] == 1 and data[i] == -1:
                couple1N1 += 1
            elif data[i - 1] == -1 and data[i] == 1:
                coupleN11 += 1
    couple00 = couple00/len(data)
    couple01 = couple01/len(data)
    couple11 = couple11/len(data)
    couple10 = coupleN10/len(data)
    coupleN10 = coupleN10/len(data)
    couple0N1 = couple0N1/len(data)
    coupleN1N1 = coupleN1N1/len(data)
    coupleN11 = coupleN11/len(data)
    couple1N1 = couple1N1/len(data)
    indicecouple.append(["[0,0]",couple00])
    indicecouple.append(["[0,1]",couple01])
    indicecouple.append(["[1,1]",couple11])
    indicecouple.append(["[1,0]",couple10])
    indicecouple.append(["[-1,0]",coupleN10])
    indicecouple.append(["[0,-1]",couple0N1])
    indicecouple.append(["[-1,-1]",coupleN1N1])
    indicecouple.append(["[-1,1]",coupleN11])
    indicecouple.append(["[1,-1]",couple1N1])
    return indicecouple

In [14]:
def printcouple(serie):
    probcouple = stvoisin(serie)
    for i in range(len(probcouple)):
        print("La probabilité d'avoir le couple  ", probcouple[i][0], " est de : ", probcouple[i][1])
print("----------PARIS-------------")
printcouple(serieParis)
print("----------NewYork-------------")
printcouple(serieNY)
print("----------Francfort-------------")
printcouple(serieFC)
print("----------HI Tech-------------")
printcouple(serieHT)

----------PARIS-------------
La probabilité d'avoir le couple   [0,0]  est de :  0.14942528735632185
La probabilité d'avoir le couple   [0,1]  est de :  0.12643678160919541
La probabilité d'avoir le couple   [1,1]  est de :  0.12260536398467432
La probabilité d'avoir le couple   [1,0]  est de :  0.10344827586206896
La probabilité d'avoir le couple   [-1,0]  est de :  0.10344827586206896
La probabilité d'avoir le couple   [0,-1]  est de :  0.0842911877394636
La probabilité d'avoir le couple   [-1,-1]  est de :  0.09961685823754789
La probabilité d'avoir le couple   [-1,1]  est de :  0.09195402298850575
La probabilité d'avoir le couple   [1,-1]  est de :  0.11494252873563218
----------NewYork-------------
La probabilité d'avoir le couple   [0,0]  est de :  0.15708812260536398
La probabilité d'avoir le couple   [0,1]  est de :  0.12260536398467432
La probabilité d'avoir le couple   [1,1]  est de :  0.1111111111111111
La probabilité d'avoir le couple   [1,0]  est de :  0.06130268199233716


## **fct pour calculer Pr(st|st-1) :**

In [15]:
def probsachantprecedant(serie,indice,indice2):
    nbindice = 0
    nbindice2 = 0
    nb = 0
    nbautre = 0
    for i in range(len(serie)):
        nb +=1
        if i != 0:
            if serie[i-1] == indice :
                nbindice +=1;
                if serie[i]== indice2 :
                     nbindice2 += 1
    pindice = nbindice/len(serie)
    pindice2 = nbindice2/nbindice
    #interres = pindice*pindice2
    # print("P (",indice ,") : ", pindice)
    print("P(",indice2,") sachant ",indice, "est égale à :",pindice2)

In [35]:
def printsachant(serie,fct):
    indice = [-1,0,1]
    for i in indice :
        print("----------------------------------------")
        for j in indice :
            fct(serie,i,j)

In [36]:
print("----------------PARIS----------------")           
printsachant(serieParis,probsachantprecedant)
print("----------------NewYork----------------")
printsachant(serieNY,probsachantprecedant)
print("----------------Francfort----------------")
printsachant(serieFC,probsachantprecedant)
print("----------------HI Tech----------------")
printsachant(serieHT,probsachantprecedant)

----------------PARIS----------------
----------------------------------------
P( -1 ) sachant  -1 est égale à : 0.33766233766233766
P( 0 ) sachant  -1 est égale à : 0.35064935064935066
P( 1 ) sachant  -1 est égale à : 0.3116883116883117
----------------------------------------
P( -1 ) sachant  0 est égale à : 0.23404255319148937
P( 0 ) sachant  0 est égale à : 0.4148936170212766
P( 1 ) sachant  0 est égale à : 0.35106382978723405
----------------------------------------
P( -1 ) sachant  1 est égale à : 0.33707865168539325
P( 0 ) sachant  1 est égale à : 0.30337078651685395
P( 1 ) sachant  1 est égale à : 0.3595505617977528
----------------NewYork----------------
----------------------------------------
P( -1 ) sachant  -1 est égale à : 0.38461538461538464
P( 0 ) sachant  -1 est égale à : 0.20512820512820512
P( 1 ) sachant  -1 est égale à : 0.41025641025641024
----------------------------------------
P( -1 ) sachant  0 est égale à : 0.1797752808988764
P( 0 ) sachant  0 est égale à : 0.

## **fct pour calculer Pr(st+1 | st) :**

In [17]:
def probsachantprochain(serie,indice,indice2):
    nbindice = 0
    nbindice2 = 0
    nb = 0
    nbautre = 0
    for i in range(len(serie)):
        nb +=1
        if i != 0:
            if serie[i] == indice :
                nbindice +=1;
                if serie[i-1]== indice2 :
                     nbindice2 += 1
    pindice = nbindice/len(serie)
    pindice2 = nbindice2/nbindice
    #interres = pindice*pindice2
    # print("P (",indice ,") : ", pindice)
    print("P(",indice2,") sachant ",indice, "est égale à :",pindice2)

In [37]:
print("----------------PARIS----------------")           
printsachant(serieParis,probsachantprochain)
print("----------------NewYork----------------")
printsachant(serieNY,probsachantprochain)
print("----------------Francfort----------------")
printsachant(serieFC,probsachantprochain)
print("----------------HI Tech----------------")
printsachant(serieHT,probsachantprochain)

----------------PARIS----------------
----------------------------------------
P( -1 ) sachant  -1 est égale à : 0.3333333333333333
P( 0 ) sachant  -1 est égale à : 0.28205128205128205
P( 1 ) sachant  -1 est égale à : 0.38461538461538464
----------------------------------------
P( -1 ) sachant  0 est égale à : 0.2903225806451613
P( 0 ) sachant  0 est égale à : 0.41935483870967744
P( 1 ) sachant  0 est égale à : 0.2903225806451613
----------------------------------------
P( -1 ) sachant  1 est égale à : 0.2696629213483146
P( 0 ) sachant  1 est égale à : 0.3707865168539326
P( 1 ) sachant  1 est égale à : 0.3595505617977528
----------------NewYork----------------
----------------------------------------
P( -1 ) sachant  -1 est égale à : 0.379746835443038
P( 0 ) sachant  -1 est égale à : 0.20253164556962025
P( 1 ) sachant  -1 est égale à : 0.4177215189873418
----------------------------------------
P( -1 ) sachant  0 est égale à : 0.18181818181818182
P( 0 ) sachant  0 est égale à : 0.46590

# 6. Calculez, pour chaque couple d’indices : <img src="q6.png">

## **Fct pour calculer Pr(st(i),st(j)) :**

In [20]:
def stvoisin(data1,data2):
    couple00 = 0
    couple01 = 0
    couple11 = 0
    couple10 = 0
    coupleN10 = 0
    couple0N1 = 0
    coupleN1N1 = 0
    coupleN11 = 0
    couple1N1 = 0
    indicecouple = []
    for i in range (len(data1)):
        if i != 0 :
            if data1[i - 1] == 0 and data2[i] == 0:
                couple00 += 1
            elif data1[i - 1] == 0 and data2[i] == 1:
                couple01 += 1
            elif data1[i - 1] == 1 and data2[i] == 1:
                couple11 += 1
            elif data1[i - 1] == 1 and data2[i] == 0:
                couple10 += 1
            elif data1[i - 1] == 0 and data2[i] == -1:
                couple0N1 += 1
            elif data1[i - 1] == -1 and data2[i] == -1:
                coupleN1N1 += 1
            elif data1[i - 1] == -1 and data2[i] == 0:
                coupleN10 += 1
            elif data1[i - 1] == 1 and data2[i] == -1:
                couple1N1 += 1
            elif data1[i - 1] == -1 and data2[i] == 1:
                coupleN11 += 1
    couple00 = couple00/len(data1)
    couple01 = couple01/len(data1)
    couple11 = couple11/len(data1)
    couple10 = coupleN10/len(data1)
    coupleN10 = coupleN10/len(data1)
    couple0N1 = couple0N1/len(data1)
    coupleN1N1 = coupleN1N1/len(data1)
    coupleN11 = coupleN11/len(data1)
    couple1N1 = couple1N1/len(data1)
    indicecouple.append(["[0,0]",couple00])
    indicecouple.append(["[0,1]",couple01])
    indicecouple.append(["[1,1]",couple11])
    indicecouple.append(["[1,0]",couple10])
    indicecouple.append(["[-1,0]",coupleN10])
    indicecouple.append(["[0,-1]",couple0N1])
    indicecouple.append(["[-1,-1]",coupleN1N1])
    indicecouple.append(["[-1,1]",coupleN11])
    indicecouple.append(["[1,-1]",couple1N1])
    return indicecouple

In [38]:
def printcouple2(serie1,serie2):
    probcouple = stvoisin(serie1,serie2)
    for i in range(len(probcouple)):
        print("La probabilité d'avoir le couple  ", probcouple[i][0], " est de : ", probcouple[i][1])

In [41]:
print("Pr(PARIS|NEWYORK) :")
printcouple2(serieParis,serieNY)
print("_________________________________________________________")
print("Pr(NEWYORK|PARIS) :")
printcouple2(serieNY,serieParis)

Pr(PARIS|NEWYORK) :
La probabilité d'avoir le couple   [0,0]  est de :  0.16091954022988506
La probabilité d'avoir le couple   [0,1]  est de :  0.13026819923371646
La probabilité d'avoir le couple   [1,1]  est de :  0.10344827586206896
La probabilité d'avoir le couple   [1,0]  est de :  0.07279693486590039
La probabilité d'avoir le couple   [-1,0]  est de :  0.07279693486590039
La probabilité d'avoir le couple   [0,-1]  est de :  0.06896551724137931
La probabilité d'avoir le couple   [-1,-1]  est de :  0.09961685823754789
La probabilité d'avoir le couple   [-1,1]  est de :  0.12260536398467432
La probabilité d'avoir le couple   [1,-1]  est de :  0.13409961685823754
_________________________________________________________
Pr(NEWYORK|PARIS) :
La probabilité d'avoir le couple   [0,0]  est de :  0.15708812260536398
La probabilité d'avoir le couple   [0,1]  est de :  0.09961685823754789
La probabilité d'avoir le couple   [1,1]  est de :  0.14942528735632185
La probabilité d'avoir le couple

In [42]:
print("Pr(FC|HT) :")
printcouple2(serieFC,serieHT)
print("_________________________________________________________")
print("Pr(HT|FC) :")
printcouple2(serieHT,serieFC)

Pr(FC|HT) :
La probabilité d'avoir le couple   [0,0]  est de :  0.11494252873563218
La probabilité d'avoir le couple   [0,1]  est de :  0.1417624521072797
La probabilité d'avoir le couple   [1,1]  est de :  0.13026819923371646
La probabilité d'avoir le couple   [1,0]  est de :  0.04980842911877394
La probabilité d'avoir le couple   [-1,0]  est de :  0.04980842911877394
La probabilité d'avoir le couple   [0,-1]  est de :  0.09195402298850575
La probabilité d'avoir le couple   [-1,-1]  est de :  0.08045977011494253
La probabilité d'avoir le couple   [-1,1]  est de :  0.15708812260536398
La probabilité d'avoir le couple   [1,-1]  est de :  0.12260536398467432
_________________________________________________________
Pr(HT|FC) :
La probabilité d'avoir le couple   [0,0]  est de :  0.11877394636015326
La probabilité d'avoir le couple   [0,1]  est de :  0.08045977011494253
La probabilité d'avoir le couple   [1,1]  est de :  0.18007662835249041
La probabilité d'avoir le couple   [1,0]  est de 

## **Fct pour calculer Pr(st(i)|st(j)) :**

In [22]:
def probsachantdata(serie,serie2,indice,indice2):
    nbindice = 0
    nbindice2 = 0
    nb = 0
    nbautre = 0
    for i in range(len(serie)):
        nb +=1
        if i != 0:
            if serie[i] == indice :
                nbindice +=1;
                if serie2[i]== indice2 :
                     nbindice2 += 1
    pindice = nbindice/len(serie)
    pindice2 = nbindice2/nbindice
    #interres = pindice*pindice2
    # print("P (",indice ,") : ", pindice)
    print("P(",indice2,") sachant ",indice, "est égale à :",pindice2)

In [45]:
def printsachant2(serie1,serie2,fct):
    indice = [-1,0,1]
    for i in indice :
        print("----------------------------------------")
        for j in indice :
            fct(serie1,serie2,i,j)

In [47]:
print("----------------PARIS|NY----------------")           
printsachant2(serieParis,serieNY,probsachantdata)
print("----------------NeY|FC----------------")
printsachant2(serieNY,serieFC, probsachantdata)
print("----------------FC|NY----------------")
printsachant2(serieFC,serieNY,probsachantdata)
print("----------------HI Tech|NY----------------")
printsachant2(serieHT,serieNY,probsachantdata)

----------------PARIS|NY----------------
----------------------------------------
P( -1 ) sachant  -1 est égale à : 0.6153846153846154
P( 0 ) sachant  -1 est égale à : 0.24358974358974358
P( 1 ) sachant  -1 est égale à : 0.14102564102564102
----------------------------------------
P( -1 ) sachant  0 est égale à : 0.1935483870967742
P( 0 ) sachant  0 est égale à : 0.5376344086021505
P( 1 ) sachant  0 est égale à : 0.26881720430107525
----------------------------------------
P( -1 ) sachant  1 est égale à : 0.14606741573033707
P( 0 ) sachant  1 est égale à : 0.21348314606741572
P( 1 ) sachant  1 est égale à : 0.6404494382022472
----------------NeY|FC----------------
----------------------------------------
P( -1 ) sachant  -1 est égale à : 0.569620253164557
P( 0 ) sachant  -1 est égale à : 0.25316455696202533
P( 1 ) sachant  -1 est égale à : 0.17721518987341772
----------------------------------------
P( -1 ) sachant  0 est égale à : 0.2159090909090909
P( 0 ) sachant  0 est égale à : 0.5

# 8. Qu’est-ce que vous pouvez déduire en comparant ces probabilités ?

Je remarque que souvent les probabilités d'un agmentation ou de baisse des bourses sont coordonnée
Par exemple on peut voir que si la bourse de New York augmente, celle de Francfort a une probabilité de 0.60 d'augementer egalement

La probabilité est la même pour une baisse ou une stagnation de la bourse

Cela s'applique au 4 dataset que j'ai pu analyser.

# 9. Comment pensez-vous qu’un agent intelligent pourrait les utiliser pour gagner de l’argent en achetant et vendant ces indices ?

Je pense qu'un agent intelligent peut grace a ces chiffres prédire la probabilité qu'il ait une chute ou un hausse des bourses. 
On peut voir que parfois les probabilités d'une evnement en finction d'un autre est très haut donc ca peut permettre a l'agent de prédire la meilleure action a faire.